# CALLING GOOGLE VISION API

In [1]:
#install google cloud vision

#!pip install google-cloud-vision
#!pip install google-cloud-storage
#import dependencies
from google.cloud import vision
from google.cloud import storage
#from google.cloud.vision_v1 import enums
from google.cloud.vision_v1 import ImageAnnotatorClient
from google.cloud.vision_v1 import types
import os
import json
from datetime import datetime 
import csv

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="awesome-tensor-417112-23175d33a089.json" 

#awesome-tensor-417112-23175d33a089.json --> project key

In [2]:
def detect_text_edited(path):
    """Detects text in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
        
    return texts

# CREATING LIST WITH BLOCKS OF FULL NO. OF WORDS 

In [3]:
def string_to_stringList(firstEntry):
    
    temp = ''
    test_list=[]
    
    for i in range(len(firstEntry)):
        if(firstEntry[i]!= str('\n') ):
            temp += firstEntry[i]
            if(i+1==len(firstEntry)):
                test_list.append(temp)
        else:
            test_list.append(temp)
            i+=1
            temp=''
            
    return test_list
    

# SLUICE BOX FUNCTIONS

----------------
**1. Check for Bank Name**

----------------

In [4]:
def check_bank_name(currentWord, list):
    if ("بنك" in currentWord or "bank" in currentWord.lower()) and not(currentWord in list):
        list.append(currentWord)

----------------
**2. Check for Amount**

----------------

In [5]:
def check_amount_paid (currentWord, list):
    if currentWord[0]>='٠' and currentWord[0]<='٩':
        list.append(currentWord)

----------------
**3. Check for Currency**

----------------

In [6]:
def check_currency(currentWord, list):

    currency_list_en = ['egp', 'usd', 'eur', 'gbp', 'sar', 'kwd', 'jod', '$', 'us']              
    currency_list_ar = ['جنيه', 'جنية' , 'دولار', 'دينار', 'ريال', 'درهم']

    for curr in currency_list_en:
        if curr in currentWord.lower():
            list.append(currentWord)
            break
    if not(currentWord in list): 
        for curr in currency_list_ar:
            if curr in currentWord:
                list.append(currentWord)
                break


----------------
**4. Check for magnetic ink line**

----------------

In [7]:
def check_micr_line(currentWord, list):
    micr_symbols = ['⑀', '⑁', '⑂', '⑄', '⑅', '⑆', '⑇', '⑈', '⑉', '⑊']

    for micr_symbol in micr_symbols:
        if micr_symbol in currentWord.lower():
            list.append(currentWord)
            break


----------------
**5. Check for payee name**

----------------

In [8]:
def check_payee_name(currentWord, listBank, listAmount, listBranch, listName):

    if currentWord[0]>='أ' and currentWord[0]<= 'ي':
        if "بنك" in currentWord and not(currentWord in listBank):
            listBank.append(currentWord) 
                
        elif "فقط" in currentWord or "لا غير" in currentWord and not(currentWord in listAmount):
            listAmount.append(currentWord) 
            
        elif "فرع" in currentWord and not(currentWord in listBranch):
            listBranch.append(currentWord)

        elif not("قدره" in currentWord or "مبلغ" in currentWord or "قدرة" in currentWord):
            listName.append(currentWord)


----------------
**6. Check for bank address**

----------------

In [9]:
def check_branch_address(currentWord, list):

    governorates_list = ['cairo', 'giza', 'alexandria', 'dakahlia', 'red sea',
                'beheira', 'fayoum', 'gharbiya', 'ismailia', 'menofia',
                'Minya', 'qaliubiya', 'new valley', 'suez', 'aswan',
                'assiut', 'beni suef', 'port said', 'damietta', 'sharkia', 
                'south sinai', 'kafr al sheikh', 'matrouh', 'luxor', 'qena', 
                'north sinai', 'sohag']

    for governorate in governorates_list:
            if governorate in currentWord.lower() or ' st' in currentWord.lower() or 'street' in currentWord.lower() or "branch" in currentWord.lower() or "فرع" in currentWord:
                list.append(currentWord)
                break       

# FINAL FUNCTION 

-----------------------------------------------

--------------------------------------------

In [10]:
def checkScan(path, csv_path):
    texts = detect_text_edited(path) 
    blocks_list = string_to_stringList(texts[0].description)

    blocks_list #In memory (15:18)

    #sluice boxes(lists)
    bank_name_list = []
    branch_address_list = []
    amount_list = []
    currency_list = []
    micr_line_list = []
    payee_name_list = []

    #sluicing
    for block in blocks_list:
        check_payee_name(block, bank_name_list, amount_list, branch_address_list, payee_name_list)
        check_micr_line(block, micr_line_list)
        check_currency(block, currency_list)
        check_amount_paid(block, amount_list)
        check_branch_address(block, branch_address_list)
        check_bank_name(block, bank_name_list)
        
    #writing to csv
      #for aesthetic purposes
    
    
    if 'Bank Name: ' not in bank_name_list:
        bank_name_list.insert(0, 'Bank Name: ')
    if 'Branch Address: ' not in bank_name_list:
        branch_address_list.insert(0, 'Branch Address: ')
    if 'Amount Paid: ' not in bank_name_list:
        amount_list.insert(0, 'Amount Paid: ')
    if 'Currency: ' not in bank_name_list:
        currency_list.insert(0, 'Currency: ')
    if 'MICR Line: ' not in bank_name_list:
        micr_line_list.insert(0, 'MICR Line: ')
    if 'Payee Name: ' not in bank_name_list:
        payee_name_list.insert(0, 'Payee Name: ')
        
    #Scan Timestamp
    timeStamp = [['Date and Time of this Scan: ']]
    for val in timeStamp:
        current_date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
        val.append(current_date_time) 
        
    #writing to csv   
    with open(csv_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(val)
        writer.writerow(['  '])
        writer.writerow(bank_name_list) 
        writer.writerow(branch_address_list)
        writer.writerow(payee_name_list)
        writer.writerow(amount_list)
        writer.writerow(currency_list)
        writer.writerow(micr_line_list)
        writer.writerow(['  '])
        writer.writerow(['Check Scan Complete! ............................................................'])
        writer.writerow(['  '])


-----------------------------------------------
Calling Function Below


--------------------------------------------

# BEGINNING OF FRONT END

In [11]:
#!pip install tk-tools
#!pip install pillow

In [12]:
import tkinter as tk
from tkinter import Label
from tkinter import filedialog
from PIL import Image, ImageTk
import ctypes

In [13]:
def popupmsg():
    popup = tk.Tk()
    popup.wm_title("Success!")
    popup.geometry("250x100")
    label = tk.Label(popup, text="Checks Scanned Successfully!", font= "Helvetica 10")
    label.pack(side="top", fill="x", pady=10)
    B1 = tk.Button(popup, text="Okay", command = popup.destroy)
    B1.pack()
    popup.eval('tk::PlaceWindow . center')
    
    myappid = 'mycompany.myproduct.subproduct.version' # arbitrary string
    ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)
    popup.iconbitmap("Check-Scan-Logo.ico")
    
    
    popup.mainloop()

In [14]:
# image uploader function
def imageUploader():
    fileTypes = [("Image files", "*.png;*.jpg;*.jpeg")]
    path = tk.filedialog.askopenfilename(filetypes=fileTypes, multiple = 'true')
    
    for f in path:
        checkScan(f, "CheckScan_Results.csv")
    popupmsg()
    
    # if file is selected
    if len(path):
        img = Image.open(path)
 
    # if no file is selected, then we are displaying below message
    else:
        print("No file is Choosen !! Please choose a file.")
 
 

In [15]:
def center_screen():
    """ gets the coordinates of the center of the screen """
    global screen_height, screen_width, x_cordinate, y_cordinate
    screen_width = app.winfo_screenwidth()
    screen_height = app.winfo_screenheight()
    # Coordinates of the upper left corner of the window to make the window appear in the center
    x_cordinate = int((screen_width/2) - (window_width/2))
    y_cordinate = int((screen_height/2) - (window_height/2))
    app.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))

In [16]:

 # Main method --> The ONLY cell you need to run the program
if __name__ == "__main__":
 
    # defining tkinter object
    app = tk.Tk()
 
    # setting title and basic size to our App and centering the screen
    app.title("Check Scan")
   
    
    #change the app icon
    myappid = 'mycompany.myproduct.subproduct.version' # arbitrary string
    ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)
    app.iconbitmap("Check-Scan-Logo.ico")
  
    
    #Adjusting size and location of window
    window_height = 260
    window_width = 870
    center_screen()
    
    #Changing the background color
    app.configure(bg="white")
    
    
    #Adds the instructions for the user
    Label_heading = tk.Label(app, bg = "white",font = "Helvetica 16 bold italic", text ='CHECK SCAN')
    Label_heading.place(relx = 0.5, rely = 0.1, anchor = 'n')
    
    Label_body = tk.Label(app,bg = "white", font = "Helvetica 10" , text ='Welcome to Check Scan! To scan your checks, click the button below and select the images of the checks that you want to have scanned.')
    Label_body.place(relx = 0.5, rely = 0.35, anchor = 'center')
     
    #Adds the button style
    app.option_add("*Button*Background", "green")

 
    # defining our upload button and calls the image uploader function
    uploadButton = tk.Button(app, font = "Helvetica 10 bold", fg = "white", text="Upload Check Images", command=imageUploader)
    uploadButton.pack(side=tk.BOTTOM, pady=50)
 
    app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\mahmo\anaconda3\Lib\site-packages\PIL\Image.py", line 3231, in open
    fp.seek(0)
    ^^^^^^^
AttributeError: 'tuple' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mahmo\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\mahmo\AppData\Local\Temp\ipykernel_17752\2621853229.py", line 12, in imageUploader
    img = Image.open(path)
          ^^^^^^^^^^^^^^^^
  File "C:\Users\mahmo\anaconda3\Lib\site-packages\PIL\Image.py", line 3233, in open
    fp = io.BytesIO(fp.read())
                    ^^^^^^^
AttributeError: 'tuple' object has no attribute 'read'
